# Model-View-ViewModel (MVVM) - Podstawy - Kotlin

W tej aplikacji przyjrzymy się zastosowaniu architektury **MVVM**.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExdmllMXdiMmd0OW8wNGs3bXJucXJxYmNtZGpheGtueG5wbzZnZDduaCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/ff1yWpIVOu1FqfQA3Q/giphy.gif" width="200" />

**Model-View-ViewModel (MVVM)** to wzorzec architektoniczny stosowany w aplikacjach mobilnych (i nie tylko), który ma na celu oddzielenie logiki biznesowej od warstwy prezentacji.

W MVVM występują trzy główne komponenty:
- **Model** - Reprezentuje dane i logikę biznesową aplikacji. Model może zawierać struktury danych, metody dostępu do bazy danych, usługi sieciowe itp.
- **View** - Odpowiada za prezentację interfejsu użytkownika (UI). Może to być np. widok, przyciski, pola tekstowe, komponenty, ekrany itp. 
- **ViewModel** - Pośredniczy między Modelem a Widokiem. ViewModel zawiera dane, które mają być wyświetlane w widoku oraz metody, które obsługują interakcje użytkownika. ViewModel nie jest bezpośrednio zależny od Widoku - jest odizolowany od konkretnego interfejsu użytkownika.

Kilka korzyści wynikających z zastosowania wzorca MVVM:
- **Separacja logiki biznesowej od warstwy prezentacji** - Dzięki MVVM można utrzymać czytelność i organizację kodu, ponieważ logika biznesowa jest oddzielona od kodu odpowiedzialnego za wygląd i interakcję z użytkownikiem.
- **Testowalność** - Dzięki oddzieleniu ViewModelu od Widoku można łatwiej testować logikę biznesową, ponieważ ViewModel **nie jest bezpośrednio zależny** od interfejsu użytkownika. Można napisać testy jednostkowe dla ViewModelu, symulując różne scenariusze bez potrzeby uruchamiania całej aplikacji.
- **Ponowne wykorzystanie kodu** Dzięki MVVM można łatwiej ponownie wykorzystywać ViewModel w różnych miejscach aplikacji. Na przykład, ten sam ViewModel może być używany w różnych ekranach, które mają podobne wymagania, ale różnią się układem i wyglądem.
- **Łatwiejsze zarządzanie stanem** MVVM pomaga w zarządzaniu stanem aplikacji poprzez wprowadzenie jednokierunkowego przepływu danych. ViewModel dostarcza dane do Widoku, które je wyświetla, a wszelkie zmiany wprowadzane przez użytkownika są przekazywane z powrotem do ViewModelu.
- **Szybsze budowanie aplikacji** - Elementy MVVM są niezależne. Zmiany w pojedynczym elemencie wymagają preezebudowania tylko tego elementu, przykładowo, zmiana w klasie `ViewModel` będzie wymagała przekompilowania tylko tej klasy, a nie całego projektu.
- **Jednokierunkowy przepływ danych** - *Unidirectional data flow* (UDF) to wzorzec projektowy, w którym stan przepływa w dół, a zdarzenia przepływają w górę. Poprzez stosowanie jednokierunkowego przepływu danych, można odseparować komponenty odpowiedzialne za wyświetlanie stanu w interfejsie użytkownika od części aplikacji, które przechowują i zmieniają stan.

<img src="https://manuelvivo.dev/assets/images/2022-06-01-viewmodel-events-antipatterns-1.png" width="200" />

Wszystkie elementy architektury znamy z poprzednich przykładów, teraz napiszemy prostą aplikację wyświetlającą imiona i nazwiska użytkowników, ktorzy będą przechowywani w liście. Wykorzystamy również *dummy data* do zainicjowania listy.

### Model

Rozpocznijmy od modelu.

In [ ]:
data class User(val firstName: String, val lastName: String)

Mamy klasę `User` z dwoma polami reprezentującymi imię i nazwisko. Dodajmy `DataProvider` z danymi inicjującymi.

In [ ]:
object DataProvider {

    private val firstNames = listOf(
        "Adam", "Ewa", "Jan", "Anna", "Piotr", "Maria", "Tomasz", "Małgorzata", "Krzysztof", "Alicja",
        "Andrzej", "Joanna", "Michał", "Barbara", "Kamil", "Magdalena", "Robert", "Monika", "Mateusz", "Natalia"
    )

    private val lastNames = listOf(
        "Nowak", "Kowalski", "Wiśniewski", "Wójcik", "Kowalczyk", "Kamiński", "Lewandowski", "Zieliński", "Szymański",
        "Woźniak", "Dąbrowski", "Kozłowski", "Jankowski", "Mazur", "Kwiatkowski", "Krawczyk", "Piotrowski", "Grabowski",
        "Nowakowski", "Pawłowski"
    )

    val users = (0..40).map { User(firstNames.random(), lastNames.random())}
}

### ViewModel

W klasie `UserViewModel` przechowujemy listę wszystkich użytkowników jako `MutableLiveData<List<User>>`.

Tutaj napotykamy pewien problem, `MutableLiveData` przechowuje i daje możliwość obserwacji samej listy `List<User>`, ale nie jej elementów. Więc dodając nowy element chcemy utrworzyć nową listę i zastąpić tą przechowywaną w zmiennej `_userList`.

In [ ]:
class UserViewModel : ViewModel() {
    private var _usersList = MutableLiveData<List<User>>()
    val usersList: LiveData<List<User>>
        get() = _usersList

    init {
        reinitialize()
    }

    fun addUser(user: User){
        val currentList = _usersList.value.orEmpty().toMutableList()
        currentList.add(user)
        currentList.sortBy { it.lastName }
        _usersList.value = currentList
    }

    fun reinitialize(){
        val sortedUsers = DataProvider.users.sortedWith(compareBy(User::lastName, User::firstName))
        _usersList.value = sortedUsers
    }

    fun clear(){
        _usersList.value = emptyList()
    }
}

- `val usersList: List<User> get() = _usersList` - lista użytkowników, do której można uzyskać dostęp tylko do odczytu.
- `fun addUser(user: User)` - Metoda służy do dodawania nowego użytkownika do listy. 
- `fun reinitialize()` - Metoda służy do zresetowania zawartości listy użytkowników do wartości domyślnych.
- `fun clear()` - Metoda służy do wyczyszczenia całej listy użytkowników.

### View

Dodajmy `View` do aplikacji, czyli zdefiniujmy interfejs użytkownika. Dodajmy layouty dla aktywności, fragmentu oraz pojedynczego elementu listy.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".ui.MainActivity">

    <androidx.fragment.app.FragmentContainerView
        android:id="@+id/fragment_main"
        android:name="com.example.mvvmbasicskotlin.ui.fragment.UsersFragment"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />
</androidx.constraintlayout.widget.ConstraintLayout>

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical">

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <LinearLayout
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:layout_margin="4dp"
            android:orientation="horizontal">

            <com.google.android.material.textfield.TextInputLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="First Name">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/firstNameEditText"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:fontFamily="sans-serif"
                    android:textSize="15sp" />
            </com.google.android.material.textfield.TextInputLayout>
        </LinearLayout>

        <LinearLayout
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:layout_margin="4dp"
            android:orientation="horizontal">

            <com.google.android.material.textfield.TextInputLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="Last Name">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/lastNameEditText"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:fontFamily="sans-serif"
                    android:textSize="15sp" />
            </com.google.android.material.textfield.TextInputLayout>
        </LinearLayout>

        <Button
            android:id="@+id/addButton"
            android:layout_width="wrap_content"
            android:layout_gravity="center"
            android:text="ADD"
            android:layout_margin="4dp"
            android:layout_height="wrap_content"/>
    </LinearLayout>

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/rvList"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:layout_margin="4dp"/>

    <Button
        android:id="@+id/clearButton"
        android:layout_width="match_parent"
        android:layout_gravity="center"
        android:layout_marginStart="8dp"
        android:layout_marginEnd="8dp"
        android:text="CLEAR"
        android:layout_height="wrap_content"/>

    <Button
        android:id="@+id/resetButton"
        android:layout_width="match_parent"
        android:layout_gravity="center"
        android:layout_marginStart="8dp"
        android:layout_marginEnd="8dp"
        android:layout_marginBottom="8dp"
        android:text="RESET"
        android:layout_height="wrap_content"/>

</LinearLayout>

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:orientation="horizontal">

    <TextView
        android:id="@+id/wordTextView"
        android:layout_width="0dp"
        android:layout_weight="1"
        android:layout_height="wrap_content"
        android:layout_margin="4dp"
        android:text="slowo"
        android:textAlignment="center"
        android:layout_marginStart="8dp"
        android:textSize="24sp" />

</LinearLayout>

Zaimplementujmy elementy `RecyclerView`

In [ ]:
class UserViewHolder(private val binding: RvItemBinding) : RecyclerView.ViewHolder(binding.root) {
    fun bind(item: User) {
        val fullName = "${item.firstName} ${item.lastName}"
        binding.wordTextView.text = fullName
    }
}

In [ ]:
class UserComparator : DiffUtil.ItemCallback<User>() {
    override fun areItemsTheSame(oldItem: User, newItem: User): Boolean {
        return oldItem === newItem
    }

    override fun areContentsTheSame(oldItem: User, newItem: User): Boolean {
        return oldItem == newItem
    }
}

In [ ]:
class UserAdapter(userComparator: UserComparator) : ListAdapter<User, UserViewHolder>(userComparator) {
    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): UserViewHolder {
        return UserViewHolder(
            RvItemBinding.inflate(
                LayoutInflater.from(parent.context), parent, false
            )
        )
    }

    override fun onBindViewHolder(holder: UserViewHolder, position: Int) {
        val item = getItem(position)
        holder.bind(item)
    }
}

Następnie przejdźmy do `UsersFraggment`

Potrzebujemy dwie wartości przechowujące instancję `UserViewModel` oraz `UserAdapter`

In [ ]:
private val viewModel: UserViewModel by viewModels()
private val userAdapter by lazy { UserAdapter(UserComparator())}

Dodajmy fragment jako obserwator, oraz podepnijmy odpowiednie metody do elementów layoutu.

In [ ]:
override fun onCreateView(
    inflater: LayoutInflater, container: ViewGroup?,
    savedInstanceState: Bundle?
): View {
    binding = FragmentUsersBinding.inflate(inflater)


    viewModel.usersList.observe(viewLifecycleOwner) { users ->
        userAdapter.submitList(users.toMutableList())
    }

    binding.rvList.apply{
        adapter = userAdapter
        layoutManager = LinearLayoutManager(requireContext())
    }

    binding.addButton.setOnClickListener { onAddUser() }
    binding.resetButton.setOnClickListener { onResetUsers() }
    binding.clearButton.setOnClickListener { onClearUsers() }

    return binding.root
}

`viewModel.usersList.observe(viewLifecycleOwner) { users -> ... }` - Ta linia kodu używa metody `observe` na polu `usersList` z klasy `UserViewModel`. Umożliwia ona reagowanie na zmiany w liście użytkowników. `viewLifecycleOwner` wskazuje, że obserwator będzie aktywny w cyklu życia fragmentu. W bloku lambdy `{ users -> ... }` następuje reakcja na zmiany w liście - przekazuje ona nową listę użytkowników do `userAdapter`.

Następnie dodajmy metody obsługujące dodanie nowego użytkownika, czyszczenie i reinicjalizację danych.

In [ ]:
private fun onAddUser() {
    val firstName = binding.firstNameEditText.text.toString().trim()
    val lastName = binding.lastNameEditText.text.toString().trim()

    if (firstName.isNotBlank() && lastName.isNotBlank()) { // sprawdzam czy pola nie są puste
        val newUser = User(firstName, lastName)            // tworzę nowego użytkownika
        viewModel.addUser(newUser)                         // dodaję do listy

        binding.firstNameEditText.text?.clear()            // czyszczę pola
        binding.lastNameEditText.text?.clear()
    }
}

private fun onResetUsers() {
    viewModel.reinitialize()
}

private fun onClearUsers() {
    viewModel.clear()
}

Możemy przetestować aplikację.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExdmllMXdiMmd0OW8wNGs3bXJucXJxYmNtZGpheGtueG5wbzZnZDduaCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/ff1yWpIVOu1FqfQA3Q/giphy.gif" width="200" />